In [6]:
# check langchain version it should be 0.3 if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.3.27
0.3.24


This hands-on exercise demonstrates a simple structured workflow for AI-assisted content creation. It walks through four key steps: generating an outline, creating a draft, collecting human feedback, and revising the content based on that feedback. The workflow simulates how humans and AI can collaborate—AI creates the initial content, while human input improves clarity, relevance, and quality. It also shows how the process can be automated or made interactive using optional user feedback.

In [7]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')


In [8]:
# The LLM 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


In [10]:

# Content creation workflow
outline_prompt = ChatPromptTemplate.from_template("""
Create a detailed outline for a blog post about: {topic}
Include 4-5 main sections and 2-3 key points for each section.

Topic: {topic}
Target Audience: {audience}

Outline:
""")

draft_prompt = ChatPromptTemplate.from_template("""
Write a blog post draft based on this outline:

OUTLINE:
{outline}

TOPIC: {topic}
TARGET AUDIENCE: {audience}
TONE: {tone}

Write a comprehensive blog post draft:
""")

review_prompt = ChatPromptTemplate.from_template("""
Review this blog post draft and provide specific feedback for improvement:

BLOG DRAFT:
{draft}

Provide constructive feedback focusing on:
1. Content structure and flow
2. Clarity and readability  
3. Engagement level
4. Areas that need more detail or examples

Feedback:
""")

revision_prompt = ChatPromptTemplate.from_template("""
Based on the feedback below, revise and improve the blog post:

ORIGINAL DRAFT:
{draft}

FEEDBACK:
{feedback}

Write the improved version of the blog post:
""")

In [12]:
# Create individual chains
outline_chain = outline_prompt | llm | StrOutputParser()
draft_chain = draft_prompt | llm | StrOutputParser() 
review_chain = review_prompt | llm | StrOutputParser()
revision_chain = revision_prompt | llm | StrOutputParser()

In [14]:
def get_human_feedback(draft, use_input=False):
    """
    Displays the draft and collects feedback.
    If `use_input=True`, user can type their own feedback.
    Otherwise, predefined feedback is returned.
    """
    print("DRAFT FOR REVIEW")
    print(draft[:500] + "..." if len(draft) > 500 else draft)

    if use_input:
        print("\nEnter feedback below (press Enter when done):")
        return input("> ")

    # Default hardcoded feedback
    return (
        "The draft has a good structure but could be improved:\n"
        "1. Add more real-world examples.\n"
        "2. Simplify technical jargon for the target audience.\n"
        "3. Include practical/actionable tips.\n"
        "4. Add a stronger introduction to hook readers."
    )

In [15]:
def content_workflow(topic, audience="general readers", tone="professional", use_feedback_input=False):
    """
    Executes content creation workflow:
    1. Generate outline
    2. Write draft
    3. Collect human feedback
    4. Apply revisions
    """
    
    outline = outline_chain.invoke({"topic": topic, "audience": audience})
    draft = draft_chain.invoke({"outline": outline, "topic": topic, "audience": audience, "tone": tone})
    feedback = get_human_feedback(draft, use_input=use_feedback_input)
    final_draft = revision_chain.invoke({"draft": draft, "feedback": feedback})

    return {
        "outline": outline,
        "initial_draft": draft,
        "feedback": feedback,
        "final_draft": final_draft,
    }

In [16]:
# Run workflow
print("CONTENT CREATION WORKFLOW STARTED")

results = content_workflow(
    topic="The Impact of AI on Content Marketing", # change this topic for testing
    audience="Marketing Professionals",
    tone="Insightful and Practical",
    use_feedback_input=False  # Change to True to allow manual feedback
)

print("\nOUTLINE")
print(results["outline"])

print("\nFEEDBACK")
print(results["feedback"])

print("\nFINAL DRAFT")
print(results["final_draft"])

CONTENT CREATION WORKFLOW STARTED
DRAFT FOR REVIEW
# The Impact of AI on Content Marketing

## Introduction

Content marketing has come a long way since its inception, evolving from simple text-based articles to a multifaceted approach that includes videos, podcasts, and interactive content. As technology advances, so too does the way we create and distribute content. Enter artificial intelligence (AI), a game-changing force that is reshaping industries across the board, including marketing. This blog post aims to explore how AI is transforming ...

OUTLINE
# Blog Post Outline: The Impact of AI on Content Marketing

## Introduction
- Brief overview of the evolution of content marketing.
- Introduction to AI and its growing role in various industries.
- Purpose of the blog post: to explore how AI is transforming content marketing strategies and practices.

## Section 1: Understanding AI in Content Marketing
- **Definition of AI in the context of content marketing**
  - Explanation of AI

In [ ]:
# sample feedback to test
"""Great draft overall! A few suggestions: Add a short real-world example to make the content more relatable.
Simplify a couple of technical terms for beginner readers.
Include a short summary or key takeaway at the end."""


In [20]:
# results = content_workflow(
#     topic="The Impact of AI on Content Marketing", # change this topic for testing
#     audience="Marketing Professionals",
#     tone="Insightful and Practical",
#     use_feedback_input=True  # input your own feedback or use above sample feedback text to test